In [1]:
import boto3
import os

In [5]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

new_bucket = "formatted-zone"

try:
    minio_client.head_bucket(Bucket=new_bucket)
except:
    minio_client.create_bucket(Bucket=new_bucket)

In [7]:
import io
import re
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "textos/"

# Llistar tots els .txt del bucket d'origen , ho fem de forma automatitzada per si en un futur
# hem de carregar més textos
paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant fitxers de text"):
        key = obj["Key"]

        # Nom del fitxer (sense path)
        filename = key.split("/")[-1]

        if not filename.lower().endswith(".txt"):
            continue

        #Llegir el contingut del fitxer original
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        text = response["Body"].read().decode("utf-8", errors="ignore")

        # Tractament automàtic del text
        text_clean = re.sub(r"[ \t]+", " ", text)          # treu espais extra
        text_clean = re.sub(r"\n\s*\n+", "\n", text_clean) # elimina línies buides
        text_clean = text_clean.strip()

        # Convertir a bytes i pujar-lo al bucket formatted-zone
        new_key = f"textos/{filename}"

        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=io.BytesIO(text_clean.encode("utf-8"))
        )

Processant fitxers de text: 100%|██████████| 1/1 [00:00<00:00,  5.95it/s]
